In [1]:
import pygame
import os,sys
import PokerModel
import time

HEIGHT = 720
WIDTH = 1280

#Global constants here
BLACK = (255,255,255)
BLACK = (0,0,0)
GREY  = (50,50,50)
RED  = (207,0,0)

images = {}
scale = .5
cardSize = (WIDTH / 7, WIDTH / 5)
buffer = 50
scores = [0,0,0,0]
state = 0

######### Initialize Pygame ###########
os.environ['SDL_VIDEO_CENTERED'] = '1' #center screen
pygame.init()
pygame.display.set_caption("Poker")
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT), 0 ,32)
    
background = pygame.image.load('img/background.jpg').convert_alpha()
cardBack = pygame.image.load('img/back.png').convert_alpha()
cardBack = pygame.transform.scale(cardBack,(int(scale * cardSize[0]), int(scale * cardSize[1])))

font = pygame.font.Font('font/CoffeeTin.ttf', 50)
loadText = font.render("Loading...", 1, BLACK)
loadSize = font.size("Loading...")
loadLoc = (WIDTH/2 - loadSize[0]/2, HEIGHT/2 - loadSize[1]/2)

font = pygame.font.Font('font/CoffeeTin.ttf',150)
font2 = pygame.font.Font('font/IndianPoker.ttf', 75)
font2.set_bold(True)
font3 = pygame.font.Font('font/IndianPoker.ttf',50)
font3.set_bold(True)

startText = font2.render("Welcome to Poker!", 1, BLACK)
startSize = font2.size("Welcome to Poker!")
startLoc = (WIDTH/2 - startSize[0]/2, buffer)

startButton = font.render(" Start ", 1, BLACK)
buttonSize =font.size(" Start ")
buttonLoc = (WIDTH/2 - buttonSize[0]/2, HEIGHT/2 - buttonSize[1]/2)

buttonRect = pygame.Rect(buttonLoc, buttonSize)
buttonRectOutline = pygame.Rect(buttonLoc, buttonSize)

poker = PokerModel.Poker(scores)

#create the new variables
cardLoc = {}
round = 0
systemExit = False
    
def start():
    deck = PokerModel.Deck()

    SCREEN.blit(background, (-320,-100))
    SCREEN.blit(loadText, loadLoc)

    pygame.display.flip()

    for card in deck:
        images[str(card)] = pygame.image.load(card.image_path).convert_alpha()
        images[str(card)] = pygame.transform.scale(images[str(card)], (int(scale * cardSize[0]), int(scale * cardSize[1])))

def play():
    global state
    global round
    global systemExit
    exitButton = font3.render(" Exit ", 1, RED)
    exitButtonSize = font3.size(" Exit ")
    exitButtonLoc = (WIDTH - 250, buffer + scale * cardSize[1]/2 - exitButtonSize[1]/2)
    exitButtonRect = pygame.Rect(exitButtonLoc, exitButtonSize)
    exitButtonRectOutline = pygame.Rect(exitButtonLoc, exitButtonSize)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit();sys.exit();return;

        #when the user clicks on a card, change its color to signify a selection has occurred
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                #create a rectangle for the mouse click and for each card.  check for intersection
                mouseRect = pygame.Rect(event.pos, (1,1))
                for index in range(len(poker.playerHand)):									#this minus thirty fixes a minor bug, do not remove
                    cardRect = pygame.Rect(cardLoc[index], (int(scale * cardSize[0]), int(scale * cardSize[1])))
                    if cardRect.colliderect(mouseRect):
                        poker.playerHand[index].selected = not poker.playerHand[index].selected
                        break

                #check if we clicked the replaceButton
                if mouseRect.colliderect(buttonRect):
                    poker.replace(poker.playerHand)
                    poker.computerReplace()
                    round += 1
                    if round == 2:
#                         state += 1
#                         results_init()
                        return
                if mouseRect.colliderect(exitButtonRectOutline):
                    systemExit = True;
                    return

            
    #display background	
    SCREEN.blit(background, (-320,-100))

    #display the player's hand
    for index in range(len(poker.playerHand)):
        if not poker.playerHand[index].selected:
            SCREEN.blit(images[str(poker.playerHand[index])], cardLoc[index])
        else:
            SCREEN.blit(cardBack, cardLoc[index])

    SCREEN.blit(exitButton, exitButtonLoc)

    #display the text
#     SCREEN.blit(youText, youLoc)
#     pygame.draw.rect(SCREEN, RED, buttonRect)
#     pygame.draw.rect(SCREEN, BLACK, buttonRectOutline, 2)
#     SCREEN.blit(replaceButton, buttonLoc)

    #display the scoreboard
#     display_scoreboard()

    pygame.display.flip()
        
def play_init():


    #setup the locations for each card in the hand
    x = 4.5 * int(scale * cardSize[0])
    youLoc = (x - 150, buffer)

    for index in range(len(poker.playerHand)):
        cardLoc[index] = (x, buffer)
        x += int(scale * cardSize[0])+10

    #setup the text that will be printed to the screen
    font = pygame.font.Font('font/IndianPoker.ttf', 25)
    font.set_bold(True)
    font2 = pygame.font.Font('font/CoffeeTin.ttf', 60)
    youText = font.render("Your Hand", 1, BLACK)
    youSize = font.size("Your Hand")

    youLoc = (cardLoc[0][0],cardLoc[0][1] - 30)#(youLoc[0], buffer + scale * cardSize[1]/2 - youSize[1]/2)

    replaceButton = font2.render(" Replace ", 1, BLACK)
    buttonSize =font2.size(" Replace ")

    buttonLoc = (x + 30, buffer + scale * cardSize[1]/2 - buttonSize[1]/2)

    buttonRect = pygame.Rect(buttonLoc, buttonSize)
    buttonRectOutline = pygame.Rect(buttonLoc, buttonSize)
    
def start_up():
    global state
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit();sys.exit()

        #when the user clicks the start button, change to the playing state
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                mouseRect = pygame.Rect(event.pos, (1,1))
                if mouseRect.colliderect(buttonRect):
                    state+=1
                    play_init()
                    return

    #draw background
    SCREEN.blit(background, (-320,-100))

    #draw welcome text
    SCREEN.blit(startText, startLoc)

    #draw the start button
    pygame.draw.rect(SCREEN, RED, buttonRect)
    pygame.draw.rect(SCREEN, BLACK, buttonRectOutline, 2)
    SCREEN.blit(startButton, buttonLoc)

    pygame.display.flip()
    
#############################################################
if __name__ == "__main__":

	Myclock = pygame.time.Clock()
	start()
# 	image_path = ('img/'+str() + '' + '.png')
# 	x = 4.5 * int(scale * cardSize[0])
# 	youLoc = (x - 150, buffer)
# 	loc = (x,buffer)

	while 1:
# 		SCREEN.blit(pygame.image.load(image_path).convert_alpha(), loc)
# 		pygame.display.flip()
		if systemExit==True:
			pygame.quit();
			break
		if state == 0:
			start_up()
		elif state == 1:
			play()
# 		elif state == 2:
# 			results()
# 		elif state == 3:
# 			new_game()
		Myclock.tick(64)

time.sleep(0.5)
exit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
